In [45]:
import os
import wget
import PyPDF2
from dotenv import load_dotenv
from langchain.text_splitter import SpacyTextSplitter
from spacy.matcher import PhraseMatcher

import spacy 
import re

load_dotenv()

from langchain.agents import load_tools, Tool
from langchain.utilities.google_search import GoogleSearchAPIWrapper
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from random import sample

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain

from langchain.prompts import PromptTemplate

from langchain.chains import LLMRequestsChain, LLMChain



# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)


google_search = GoogleSearchAPIWrapper()

def google_search_about_education(query: str) -> str:
    query = query.lower()
    query = query.replace("bulletin officiel", "")
    r = google_search._google_search_results(f"{query} filetype:pdf site:eduscol.education.fr")
    if len(r) == 0:
        return "No link"
    else:
        all_docs = []
        embeddings = HuggingFaceEmbeddings()
        vectorstore = None
        text_splitter = SpacyTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0, pipeline="fr_core_news_sm")

        for result in r[:3]:
            pdf_url = result["link"] # Take the URL of the first result
            wget.download(url=pdf_url, out="data/result.pdf")
            document_text = ""
            with open("data/result.pdf", "rb") as pdf_file:
                # creating a pdf reader object
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                for page in pdf_reader.pages:
                    # extracting text from page
                    document_text += page.extract_text()
            
            os.remove("data/result.pdf")

            texts = text_splitter.split_text(document_text)
            if vectorstore is None:
                vectorstore = FAISS.from_texts(texts, embeddings)
            else:
                vectorstore.add_texts(texts)

        docs = vectorstore.similarity_search(query, k=10)


        REFINE_PROMPT_TMPL = (
            f"Your job is to produce a final memo about: {query}\n"
            "We have provided an existing memo up to a certain point: {existing_answer}\n"
            "We have the opportunity to refine the existing memo"
            "(only if needed) with some more context below.\n"
            "------------\n"
            "{text}\n"
            "------------\n"
            "Given the new context, refine the original memo\n"
            "If the context isn't useful, return the original memo."
        )
        REFINE_PROMPT = PromptTemplate(
            input_variables=["existing_answer", "text"],
            template=REFINE_PROMPT_TMPL,
        )
        prompt_template = f"Write a concise memo about {query}"+ """using the following:


        {text}


        CONCISE SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

        chain = load_summarize_chain(llm, chain_type="refine", verbose=True, question_prompt=PROMPT, refine_prompt=REFINE_PROMPT)
        summary = chain.run(docs)
        return summary



google_search_education = Tool(
        "Google Search about education",
        google_search_about_education,
        "A wrapper around Google Search, that returns only extracts of pdf about Education from the French government. Useful for when you need to answer questions about teaching. Input should be only relevant keywords separated by a space.",
    )

tools = [google_search_education]


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("quelle activité organiser pour favoriser apprentissage trandisciplinaire ?")



> Entering new AgentExecutor chain...
 I should look for activities that promote interdisciplinary learning
Action: Google Search about education
Action Input: activité apprentissage trandisciplinaire

Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 518, which is longer than the specified 500




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise memo about activité apprentissage trandisciplinaireusing the following:


        - Mobiliser différentes ressources (physiologique, bioméc anique, 
psychologique, émotionnelle) pour agir de manière efficiente.  1 
S’approprier seul ou à plusieurs par la pratique, les méthodes et outils 
pour apprendre  


- Apprendre par l’action, l’observation, l’analyse de son activité et de celle des 
autres.

 


- Répéte r un geste pour le stabiliser et le rendre plus efficace.

 


- Utiliser  des  outils  numériques  pour  observer,  évaluer  et modifier  ses  actions.  2 
Partager des règles, assumer des rôles et des responsabilités  


- Assumer les rôles sociaux spécifiques aux différentes APSA et à la classe 
(joueur, coach, arbitre, juge, observateur, tuteur, médiateur, 
organisateur…).

 


- Comprendre, respecter et faire respecter règles et règlements.

 


- Assure

"La pédagogie par projet est une bonne façon de favoriser l'apprentissage trandisciplinaire."

In [36]:
agent.run("autonomie en CE1")



> Entering new AgentExecutor chain...
 I need to find out what autonomy means in CE1
Action: Google Search about education
Action Input: autonomie CE1

Created a chunk of size 311, which is longer than the specified 200
Created a chunk of size 208, which is longer than the specified 200
Created a chunk of size 575, which is longer than the specified 200
Created a chunk of size 339, which is longer than the specified 200
Created a chunk of size 259, which is longer than the specified 200
Created a chunk of size 506, which is longer than the specified 200
Created a chunk of size 283, which is longer than the specified 200
Created a chunk of size 236, which is longer than the specified 200
Created a chunk of size 265, which is longer than the specified 200
Created a chunk of size 210, which is longer than the specified 200
Created a chunk of size 296, which is longer than the specified 200
Created a chunk of size 244, which is longer than the specified 200
Created a chunk of size 226, which is longer than the specified 200
Created a chunk of size 285, which is longer than the specified 200
Created a chunk of size 327, which is longer tha


Observation: Si le travail en autonomie permet une meilleure prise en compte de l’hétérogénéité des élèves, il requiert 
la vigilance du professeur.

Il n’est pas envisageable que des élèves de cours préparatoire soient livrés à eux-mêmes plus d’une quinzaine de minutes.

Le professeur ne peut s’isoler avec un groupe pendant un trop long moment, laissant les autres élèves, inoccupés ou bloqués par les obstacles rencontrés.


Certaines tâches ne peuvent être réalisées sans l’aide du professeur. | Les automatismes en écriture (copie, dictée, rédaction) s’obtiennent en privilégiant 
la répétition au  travers d’entraînements très réguliers, notamment lors d’activités 
ritualisées (dictée quotidienne, autodictée, phrase du jour, « jogging d’écriture »

).




Toute séance portant sur une nouvelle notion ne doit pas faire l’économie, en guise 
d’introduction, du brassage des acquis. | Le recours à cette 
classification doit devenir automatique pour amener les élèves à réfléchir sur leurs 
é

"Pour permettre aux élèves de CE1 d'atteindre l'autonomie, le professeur doit leur offrir un environnement adapté et structuré, avec des activités variées et des entraînements réguliers qui alternent des temps dirigés et des temps de travail en autonomie. Des outils d'aide et des jeux pédagogiques peuvent également être mis en place pour faciliter l'apprentissage."

In [1]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

/home/haxxor/projects/teacher/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.02k/1.02k [00:00<00:00, 630kB/s]
Downloading:  38%|███▊      | 424M/1.11G [12:00<23:39, 485kB/s]     

KeyboardInterrupt: 

Downloading:  38%|███▊      | 425M/1.11G [12:17<23:39, 485kB/s]